In [131]:
from llama_index.readers.file import  PandasCSVReader
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
from llama_index.core.agent.workflow import FunctionAgent
from dotenv import load_dotenv
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import Settings
import os
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.tools import QueryEngineTool
from llama_index.core.workflow import Context
from llama_index.agent.openai import OpenAIAgent

In [132]:
load_dotenv()
openai_token = os.getenv("OPENAI_API_KEY")
llm = OpenAI(model="gpt-4.1", token=openai_token)
embed_model = OpenAIEmbedding(model="text-embedding-3-small", token=openai_token)

In [133]:
Settings.llm= llm
Settings.embed_model = embed_model

try:

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/ciuu"
    )
    ciuu_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [134]:
parser = PandasCSVReader(pandas_config={"encoding": "latin1"})
file_extractor = {".csv": parser} 
if not index_loaded:
    ciuu_docs = SimpleDirectoryReader(
        "./data", file_extractor=file_extractor
    ).load_data()

    ciuu_index = VectorStoreIndex.from_documents(ciuu_docs)

    ciuu_index.storage_context.persist(persist_dir="./storage/ciuu")


In [135]:
ciuu_engine = ciuu_index.as_query_engine(similarity_top_k=3, structured_output=True)

In [136]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=ciuu_engine,
        name="ciuu",
        description="use this tool to get information from the ciuu database",
    )
]

In [137]:
ciuu_agent = FunctionAgent(
    name = "ciuu",
    description="use this agent to get information from the ciuu database",
    tools=query_engine_tools,
    llm=llm,
    system_prompt="""
                    Eres un asistente experto en clasificación económica que ayuda a las personas a identificar los códigos CIIU (Clasificación Industrial Internacional Uniforme) que mejor se ajustan a la descripción de su actividad económica.

                    Cuando el usuario proporcione una descripción de su trabajo, analiza cuidadosamente el texto para entender la actividad principal. Luego, busca y selecciona los códigos CIIU más relevantes según su similitud semántica y conceptual.

                    Tu respuesta debe ser un objeto JSON válido con una lista llamada "resultados", en la que cada elemento contiene dos claves: "codigo" y "descripcion".

                    Devuelve siempre entre 1 y 3 coincidencias relevantes, ordenadas por similitud. No incluyas texto adicional, explicaciones ni comentarios fuera del JSON. Devuelve solo el JSON.

                    Formato requerido:
                    {
                    "resultados": [
                        {
                        "codigo": "0112",
                        "descripcion": "Cultivo de arroz"
                        },
                        {
                        "codigo": "0113",
                        "descripcion": "Cultivo de hortalizas, raíces y tubérculos"
                        },
                        {
                        "codigo": "0111",
                        "descripcion": "Cultivo de cereales (excepto arroz), legumbres y semillas oleaginosas"
                        }
                    ]
                    }

                    Si la descripción es ambigua o insuficiente, devuelve exactamente esto:
                    {
                    "resultados": []
                    }
                    """
)



In [141]:
handler = ciuu_agent.run("Yo vendo pollo, carne y queso en mi negocio")

response = await handler

In [142]:
print(handler)

{
"resultados": [
    {
        "codigo": "4723",
        "descripcion": "Comercio al por menor de carnes (incluye aves de corral), productos cárnicos, pescados y productos de mar, en establecimientos especializados"
    },
    {
        "codigo": "4722",
        "descripcion": "Comercio al por menor de leche, productos lácteos y huevos, en establecimientos especializados"
    }
]
}
